In [1]:
import os
import sys
os.environ["PYSPARK_SUBMIT_ARGS"]='pyspark-shell'
os.environ["PYSPARK_PYTHON"]='python3'
os.environ["SPARK_HOME"]='/opt/cloudera/parcels/SPARK2/lib/spark2/'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.6-src.zip'))
os.environ["PYSPARK_PYTHON"] = 'python3'
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.0.cloudera2
      /_/

Using Python version 3.4.3 (default, Nov 17 2016 01:08:31)
SparkSession available as 'spark'.


In [41]:
import re
import sys
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession
from datetime import datetime as dt

log_format = re.compile( 
    r"(?P<host>[\d\.]+)\s" 
    r"(?P<identity>\S*)\s" 
    r"(?P<user>\S*)\s"
    r"\[(?P<time>.*?)\]\s"
    r'"(?P<request>.*?)"\s'
    r"(?P<status>\d+)\s"
    r"(?P<bytes>\S*)\s"
    r'"(?P<referer>.*?)"\s'
    r'"(?P<user_agent>.*?)"\s*'
)

def parseLine(line):
    match = log_format.match(line)
    if not match:
        return ("", "", "", "", "", "", "" ,"")

    request = match.group('request').split()
    return (match.group('host'), match.group('time').split()[0], \
       request[0], request[1], match.group('status'), match.group('bytes'), \
        match.group('referer'), match.group('user_agent'),
        dt.strptime(match.group('time').split()[0], '%d/%b/%Y:%H:%M:%S').hour)

In [3]:
# log = sc.textFile("/data/access_logs/big_log_10000")\
#         .map(lambda x: (parseLine(x)[0], 1))\
#         .filter(lambda x: "4" in x[0])\
#         .reduceByKey(lambda x, y: x + y)\
#         .sortBy(lambda x: -x[1])\
#         .take(5)

In [48]:
log = sc.textFile("/data/access_logs/big_log")\
        .map(lambda x: parseLine(x))

In [49]:
from pyspark.sql.types import *

In [50]:
log_schema = StructType(fields=[
    StructField("ip", StringType()),
    StructField("timestamp", StringType()),
    StructField("info", StringType()),
    StructField("link", StringType()),
    StructField("status", StringType()),
    StructField("size", StringType()),
    StructField("source", StringType()),
    StructField("ua", StringType()),
    StructField("hour", IntegerType())
])

In [51]:
df = spark.createDataFrame(log, log_schema)

In [ ]:
df.show(10)

In [52]:
df.groupby("hour").count().orderBy("hour").show(24)

+----+--------+
|hour|   count|
+----+--------+
|   0|10058341|
|   1| 7170144|
|   2|  437217|
+----+--------+

